# **Run LLMUtils script**

In [1]:
import re
import pandas as pd

from typing import List

In [2]:
# make pandas dataframe fit the entire width of the page, and display all content in each cell
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)
# make rows display in full
pd.set_option("display.max_rows", None)

In [3]:
# import modules from utils.py
from utils import *
import os

In [4]:
authors = [
    "Shi Feng",
    "Zhijing Jin",
    "Jacob Foerster",
    "Elad Hazan",
    "Florian Tramèr",
    "He He",
    "Ilija Bogunovic",
    "Yoav Artzi",
    "Stefano Ermon",
    "Jacob Andreas",
    "Dylan Hadfield-Menell",
    "Pierre-Luc Bacon",
    "Yejin Choi",
    "Natasha Jacques",
    "Zico Kotler",
    "Dawn Song",
    "Yuekai Sun",
    "Scott Niekum",
    "Kyunghyun Cho",
    "Rajesh Ranganath",
    "Fei Fang",
    "Steven Wu",
    "Andrea Zanette",
    "Hima Lakkaraju",
    "Matt Fredrikson",
    "Sham Kakade",
    "Diyi Yang",
    "Micah Goldblum",
    "Yu Su",
    "Daniel Kang",
    "Pavel Izmailov",
    "Ding Zhao",
    "Tatsunori Hashimoto",
    "Sewon Min",
    "Bo Li",
    "Gagandeep Singh",
    "Lionel Levine",
    "Tanya Goyal",
    "Brad Knox",
    "Roger Grosse",
    "Chris Callison-Burch",
    "Edgar Dobriban",
    "Eric Wong",
    "Yu Zhou",
    "Sanjeev Arora",
    "Danqi Chen",
    "Boaz Barak",
    "Kiantè Brantley",
    "Sheila McIllraith",
    "Chi Jin",
    "Andreea Bobu",
    "Yoon Kim",
    "John Hewitt",
    "Tim Althoff",
    "Tim Rocktaschel",
    "Alessandro Abate",
    "Buck Shlegeris",
    "Ethan Perez",
    "Samuel R. Bowman",
    "Akbir Khan",
]

In [5]:
# Open file for reading contents

# path to txts
path = "/Users/joanvelja/Documents/neurips_posters"

# Merge all contents of txt files starting in 'session' into one raw string variable
raw = r""
for file in os.listdir(path):
    if file.startswith("session"):
        with open(os.path.join(path, file), "r") as f:
            raw += f.read()

In [6]:
# Open file for interested papers extracted via Gemini

interests = r""
for file in os.listdir(path):
    if file.startswith("interests"):
        with open(os.path.join(path, file), "r") as f:
            interests += f.read()

In [ ]:
# Regex pattern to extract paper titles and split them from shortened abstracts (abstract contained within parentheses)
# Papers are separated by a double newline character

interest_pattern = r"(.+?)\((.+?)\)\n\n"
interests_matches = re.findall(interest_pattern, interests, re.DOTALL)

# Strip leading and trailing whitespace from each tuple in the list
interests_matches = [
    (title.strip(), abstract.strip()) for title, abstract in interests_matches
]

# create a dataframe from the extracted papers
interests_df = pd.DataFrame(interests_matches, columns=["title", "abstract"])


In [8]:
interests_df["title"].tolist()

['RepLiQA: A Question-Answering Dataset for Benchmarking LLMs on Unseen Reference Content\n\nAgentBoard: An Analytical Evaluation Board of Multi-turn LLM Agents\n\nFindingEmo: An Image Dataset for Emotion Recognition in the Wild',
 'Do causal predictors generalize better to new domains?',
 'Can LLMs Implicitly Learn Numeric Parameter Constraints in Data Science APIs?\n\nRAGChecker: A Fine-grained Framework for Diagnosing Retrieval-Augmented Generation\n\nRoleAgent: Building, Interacting, and Benchmarking High-quality Role-Playing Agents from Scripts\n\nSubjECTive-QA: A dataset for the subjective evaluation of answers in Earnings Call Transcripts',
 'Evaluating Numerical Reasoning in Text-to-Image Models',
 'Benchmarking LLMs via Uncertainty Quantification\n\nSpreadsheetBench: Towards Challenging Real World Spreadsheet Manipulation\n\nBLEnD: A Benchmark for LLMs on Everyday Knowledge in Diverse Cultures and Languages\n\nEmbodied Agent Interface: Benchmarking LLMs for Embodied Decision M

In [9]:
def extract_session_info(text: str, paper_start_pos: int) -> dict | None:
    # Pattern to match session details above paper
    session_pattern = r"### Poster Session\n\nPoster Session (\d+)([A-Za-z]+)\n---------------------\n\n##### ([^\n]+)\n\n([^\n]+)"

    # Find all session headers
    sessions = list(re.finditer(session_pattern, text))

    # Find the closest session header before paper
    closest_session = None
    for session in sessions:
        if session.end() < paper_start_pos:
            closest_session = session
        else:
            break

    if closest_session:
        return {
            "session_number": closest_session.group(1),
            "session_wing": closest_session.group(2),
            "location": closest_session.group(3),
            "time": closest_session.group(4),
        }
    return None


def extract_papers(
    text: str, authors_of_interest: List[str], titles_crossref: List[str] = []
) -> pd.DataFrame:
    # Updated pattern to capture URL
    paper_pattern = (
        r"#{5}\s+\*\*\[([^\]]+)\]\(([^\)]+)\)\*\*\n\n(.*?)\n\n(.*?)(?=\n#{5}|\Z)"
    )

    papers = []

    for match in re.finditer(paper_pattern, text, re.DOTALL):
        title = match.group(1)
        url = f"www.neurips.cc{match.group(2)}"
        authors = match.group(3)
        abstract = match.group(4)

        # Get session info for this paper
        session_info = extract_session_info(text, match.start())

        if any(
            author.lower() in authors.lower() for author in authors_of_interest
        ) or any(
            title.lower() in title_crossref.lower()
            for title_crossref in titles_crossref
            if titles_crossref is not None
        ):
            paper_data = {
                "title": title,
                "url": url,
                "authors": authors,
                "abstract": abstract,
            }

            # Add session info if found
            if session_info:
                paper_data.update(
                    {
                        "session_number": session_info["session_number"],
                        "session_wing": session_info["session_wing"],
                        "location": session_info["location"],
                        "time": session_info["time"],
                    }
                )

            papers.append(paper_data)

    # Create DataFrame
    df = pd.DataFrame(papers)
    return df

In [10]:
# Extract matching papers
df = extract_papers(raw, authors)

In [11]:
df

,title,url,authors,abstract
0,NewTerm: Benchmarking Real-Time New Terms for Large Language Models with Annual Updates,www.neurips.cc/virtual/2024/poster/97724,Hexuan Deng · Wenxiang Jiao · Xuebo Liu · Min Zhang · Zhaopeng Tu,"Despite their remarkable abilities in various tasks, large language models (LLMs) still struggle with real-time information (e.g., new facts and terms) due to the knowledge cutoff in their development process. However, existing benchmarks focus on outdated content and limited fields, facing difficulties in real-time updating and leaving new terms unexplored. To address this problem, we propose an adaptive benchmark, NewTerm, for real-time evaluation of new terms. We design a highly automated construction method to ensure high-quality benchmark construction with minimal human effort, allowing flexible updates for real-time information. Empirical results on various LLMs demonstrate over 20% performance reduction caused by new terms. Additionally, while updates to the knowledge cutoff of LLMs can cover some of the new terms, they are unable to generalize to more distant new terms. We also analyze which types of terms are more challenging and why LLMs struggle with new terms, paving the way for future research. Finally, we construct NewTerm 2022 and 2023 to evaluate the new terms updated each year and will continue updating annually. The benchmark and codes can be found at https://anonymous.4open.science/r/NewTerms.\n\n \n\nPoster\n\n#5105\n"
1,TorchSpatial: A Location Encoding Framework and Benchmark for Spatial Representation Learning,www.neurips.cc/virtual/2024/poster/97807,Nemin Wu · Qian Cao · Zhangyu Wang · Zeping Liu · Yanlin Qi · Jielu Zhang · Joshua Ni · X. Yao · Hongxu Ma · Lan Mu · Stefano Ermon · Tanuja Ganu · Akshay Nambi · Ni Lao · Gengchen Mai,"Spatial representation learning (SRL) aims at learning general-purpose neural network representations from various types of spatial data (e.g., points, polylines, polygons, networks, images, etc.) in their native formats. Learning good spatial representations is a fundamental problem for various downstream applications such as species distribution modeling, weather forecasting, trajectory generation, geographic question answering, etc. Even though SRL has become the foundation of almost all geospatial artificial intelligence (GeoAI) research, we have not yet seen significant efforts to develop an extensive deep learning framework and benchmark to support SRL model development and evaluation. To fill this gap, we propose TorchSpatial, a learning framework and benchmark· for location (point) encoding, which is one of the most fundamental data types of spatial representation learning. TorchSpatial contains three key components: 1) a unified location encoding framework that consolidates 15 commonly recognized location encoders, ensuring scalability and reproducibility of the implementations; 2) the LocBench benchmark tasks encompassing 7 geo-aware image classification and 4 geo-aware image regression datasets; 3) a comprehensive suite of evaluation metrics to quantify geo-aware models’ overall performance as well as their geographic bias, with a novel Geo-Bias Score metric. Finally, we provide a detailed analysis and insights into the model performance and geographic bias of different location encoders. We believe TorchSpatial will foster future advancement of spatial representation learning and spatial fairness in GeoAI research. The TorchSpatial model framework, LocBench, and Geo-Bias Score evaluation framework are available at https://github.com/seai-lab/TorchSpatial.\n\n \n\nPoster\n\n#5109\n"
2,DataComp-LM: In search of the next generation of training sets for language models,www.neurips.cc/virtual/2024/poster/97814,Amro Abbas · Alon Albalak · Kushal Arora · Hritik Bansal · Yonatan Bitton · Yair Carmon · Khyathi Chandu · Mayee Chen · Giannis Daras · Achal Dave · Alex Dimakis · Alaaeldin El-Nouby · Fartash Faghri · Alex Fang · Samir Yitzhak Gadre · Josh 